# 🤖 Janus + Azure OpenAI Assistant
Welcome! Ask any SQL-style question about your Janus Data Mesh (Trino-based).

**Example questions:**
- How many catalogs are connected to the system?
- List all available schemas in the `sales` catalog.
- Show top 5 tables by size.

**Type `exit` to quit.**

In [ ]:
import os
from dotenv import load_dotenv
from langchain.agents import initialize_agent, Tool
from langchain_community.chat_models import AzureChatOpenAI
from langchain.chat_models import init_chat_model
from ai_tools import JanusQueryTool
from IPython.display import display, Markdown
import ipywidgets as widgets

load_dotenv()

AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["OPENAI_API_VERSION"] = "2024-02-15-preview"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-4o"

with open("ai_context.txt", "r") as f:
    ai_context = f.read()

print("Creating LLM...")
llm = init_chat_model(
    model="gpt-4o",
    model_provider="azure_openai"
)
print("LLM created")

janus_tool = JanusQueryTool()
tools = [
    Tool(
        name="JanusQueryTool",
        func=janus_tool.run,
        description="""
        Use this tool to run SQL-like queries on Janus data mesh.
        Use Trino SQL.
        The Janus data mesh contains enterprise data products.
        This is a multiple catalog setup.
        """
    )
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    handle_parsing_errors=True
)

In [ ]:
input_box = widgets.Text(
    value='',
    placeholder='Type your SQL-like data question here...',
    description='Query:',
    layout=widgets.Layout(width='100%'),
    style={'description_width': 'initial'}
)

output_box = widgets.Output()

def on_submit(change):
    query = change['new']
    if query.lower() == "exit":
        input_box.disabled = True
        return
    response = agent.run(
        f"Use this context to help you answer the query.\n\nContext:\n{ai_context}\n\nQuery:\n{query}"
    )
    with output_box:
        display(Markdown(f"**🗨️ Query:** `{query}`\n\n**💡 Response:**\n\n{response}"))
    input_box.value = ''

input_box.observe(on_submit, names='value')

display(Markdown("### 📊 Janus + Azure OpenAI Assistant"))
display(input_box, output_box)